# image_processing

In [25]:
%%bash
mni_brain=${FSLDIR}/data/standard/MNI152_T1_2mm_brain
#cd /Users/analive/Documents/Servicios_de_Salud/investigacion/brain_AI/imag_proof/Rm_Craneo_Con_Contraste_599672740/nii/
nii_file=(/Users/analive/Documents/Servicios_de_Salud/investigacion/brain_AI/imag_proof/Rm_Craneo_Con_Contraste_599672740/nii/Rm_Craneo_Con_Contraste_599672740_FLAIR_SPIR_20190705104540_401.nii.gz)
in_dir=`dirname $nii_file`
nii_file=`remove_ext $nii_file`

#== 1. # brain extracted
bet $nii_file ${nii_file}_brain -R -f 0.5 -g 0 -m  # brain extracted
nii_file=${nii_file}_brain 
mask_file=${nii_file}_mask
fslmaths $mask_file -eroF $mask_file # Spatial Filtering: Minimum filtering of all voxels
csf_thr=`fslstats $nii_file -k $mask_file -s` # mask (k) , use standart deviation (s)
fslmaths $nii_file -mas $mask_file -uthr $csf_thr -dilM -bin ${nii_file}_csf_mask # mask, data preparation and binarise
fslmaths $mask_file -mas $mask_file -sub ${nii_file}_csf_mask -bin $mask_file # Extract cerebrospinal fluid

#=== 2. spatial smoothing using a Gauss filter FWHM=2.355 sigma
fslmaths $nii_file -s 1.3 $nii_file 

#=== Registration to standard space
flirt -in $nii_file -ref $mni_brain -out ${nii_file}_mni-2mm -omat ${nii_file}_mni-2mm.mat -bins 256 -cost corratio -searchrx -90 90 -searchry -90 90 -searchrz -90 90 -dof 12  -interp trilinear

#=== 3. Apply transformation to the mask
flirt -in $mask_file -ref $mni_brain -out $mask_file -init ${nii_file}_mni-2mm.mat -applyxfm
nii_file=${nii_file}_mni-2mm


#=== 4. Splitting of both hemispheres 

# Extrae una región de interés (ROI region of interest) de una imagen.
fslroi ${nii_file} ${nii_file}_r 0 45 0 -1 0 -1 
fslroi ${nii_file} ${nii_file}_l 45 45 0 -1 0 -1
fslroi $mask_file ${mask_file}_r 0 45 0 -1 0 -1
fslroi $mask_file ${mask_file}_l 45 45 0 -1 0 -1

# Flipped
fslswapdim ${nii_file}_l -x y z ${nii_file}_l_flipped
fslswapdim ${mask_file}_l -x y z ${mask_file}_l_flipped
fslmaths ${mask_file}_r -mul ${mask_file}_l_flipped -bin ${mask_file}_no_csf



          Will use voxel-based orientation which is probably incorrect - *PLEASE CHECK*!

